In [ ]:
# 匯入所需的模組 [B]
%matplotlib inline
import time
import matplotlib.pyplot as plt
import numpy as np # Use numpy to convert to arrays
import pandas as pd # 引用套件並縮寫為 pd
import pmdarima as pm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_model import ARMAResults
import pymysql
from datetime import datetime, timedelta
from meteocalc import Temp, dew_point, heat_index, wind_chill, feels_like
from sklearn import preprocessing
# 匯入所需的模組 [E]

In [ ]:
# 參數宣告 [B]
begin_date = '2017-06-08'
end_date = '2021-06-08'
train_begin_date = '2017-06-08'
train_end_date = '2017-09-07'
test_begin_date = '2019-06-08'
test_end_date = '2020-06-07'
predict_begin_date = '2020-06-08'
predict_end_date = '2021-06-07'
# 參數宣告 [E]

In [ ]:
# 正規化最大最小值 [B]
temperature_range = np.array([0, 50])
pres_range = np.array([0, 1500])
wind_speed_range = np.array([0, 80])
wind_dir_range = np.array([0, 360])
r24_range = np.array([0, 400])
huvi_range = np.array([0, 30])
day_of_year_range = np.array([1, 366])
week_range = np.array([1, 53])
hour_range = np.array([0, 24])
minute_range = np.array([0, 60])
weekday_range = np.array([1, 7])
minutes_of_the_day_range = np.array([0, 86400])
demand_range = np.array([0, 700])
# 正規化最大最小值 [B]

In [ ]:
# 體感溫度計算 [B]
def c_feel_like(temperature, wind_speed):
    feels_like = int(13.12+(0.615*float(temperature))-(11.37*(float(wind_speed)*3.6)**0.16)+(0.3965*float(temperature))*((float(wind_speed)*3.6)**0.16))
    return feels_like
# 體感溫度計算 [E]

In [ ]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = 'ncku_demand' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `id`, `demand_min`, `demand_quarter`, `Total_value`, `Temperature`, 
                    `humd`, `pres`, `w_DSD`, `w_DIR`, `h_FX`, `24R`, `h_UVI`, `T_Min`, `T_Max`, 
                    `day_of_year`, `data_week`, `data_date`, `data_hour`, `data_minute`, `data_weekday`, 
                    `minutes_of_the_day`, `datetime`
                    FROM `demand_with_weather_data` 
                    WHERE `datetime` >= '{}' AND `datetime` < '{}'
                    ORDER BY `datetime` '''.format(begin_date, end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    timestamp = pd.to_datetime(demand_dataframe.datetime, infer_datetime_format=True).values.astype(float)
    demand_dataframe['timestamp'] = timestamp.tolist()
    demand_dataframe = demand_dataframe.set_index('datetime')
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

In [ ]:
# 補缺失值 [B]
demand_dataframe = demand_dataframe.interpolate(method="linear") 
# 補缺失值 [E]

In [ ]:
# 體感溫度計算 [B]
demand_dataframe['apparent_temp'] = demand_dataframe.apply(lambda demand_dataframe: c_feel_like(demand_dataframe['Temperature'], demand_dataframe['humd'], demand_dataframe['w_DSD']), axis=1)
# 體感溫度計算 [E]

In [ ]:
# 資料正規化 [B]
# 宣告實例 [B]
temperature_zscore = preprocessing.MinMaxScaler()
pres_zscore = preprocessing.MinMaxScaler()
wind_speed_zscore = preprocessing.MinMaxScaler()
wind_dir_zscore = preprocessing.MinMaxScaler()
max_wind_zscore = preprocessing.MinMaxScaler()
r24_zscore = preprocessing.MinMaxScaler()
huvi_zscore = preprocessing.MinMaxScaler()
apparent_temp_zscore = preprocessing.MinMaxScaler()
T_Min_zscore = preprocessing.MinMaxScaler()
T_Max_zscore = preprocessing.MinMaxScaler() 
day_of_year_zscore = preprocessing.MinMaxScaler() 
data_week_zscore = preprocessing.MinMaxScaler() 
data_hour_zscore = preprocessing.MinMaxScaler() 
data_minute_zscore = preprocessing.MinMaxScaler() 
data_weekday_zscore = preprocessing.MinMaxScaler() 
minutes_of_the_day_zscore = preprocessing.MinMaxScaler()
demand_zscore = preprocessing.MinMaxScaler() 
# 宣告實例 [E]
# 資料 reshape [B]
temperature_reshape_data = demand_dataframe['Temperature'].values.reshape(-1, 1)
pres_reshape_data = demand_dataframe['pres'].values.reshape(-1, 1)
wind_speed_reshape_data = demand_dataframe['w_DSD'].values.reshape(-1, 1)
wind_dir_reshape_data = demand_dataframe['w_DIR'].values.reshape(-1, 1)
max_wind_reshape_data = demand_dataframe['h_FX'].values.reshape(-1, 1)
r24_reshape_data = demand_dataframe['24R'].values.reshape(-1, 1)
huvi_reshape_data = demand_dataframe['h_UVI'].values.reshape(-1, 1)
apparent_temp_reshape_data = demand_dataframe['apparent_temp'].values.reshape(-1, 1)
T_Min_reshape_data = demand_dataframe['T_Min'].values.reshape(-1, 1)
T_Max_reshape_data = demand_dataframe['T_Max'].values.reshape(-1, 1)
day_of_year_reshape_data = demand_dataframe['day_of_year'].values.reshape(-1, 1)
data_week_reshape_data = demand_dataframe['data_week'].values.reshape(-1, 1)
data_hour_reshape_data = demand_dataframe['data_hour'].values.reshape(-1, 1)
data_minute_reshape_data = demand_dataframe['data_minute'].values.reshape(-1, 1)
data_weekday_reshape_data = demand_dataframe['data_weekday'].values.reshape(-1, 1)
minutes_of_the_day_reshape_data = demand_dataframe['minutes_of_the_day'].values.reshape(-1, 1)
demand_reshape_data = demand_dataframe['demand_quarter'].values.reshape(-1, 1)
# 資料 reshape [E]
# 部分計算 [B]
temperature_zscore.partial_fit(temperature_range.reshape(-1, 1))
pres_zscore.partial_fit(pres_range.reshape(-1, 1))
wind_speed_zscore.partial_fit(wind_speed_range.reshape(-1, 1))
wind_dir_zscore.partial_fit(wind_dir_range.reshape(-1, 1))
max_wind_zscore.partial_fit(wind_speed_range.reshape(-1, 1))
r24_zscore.partial_fit(r24_range.reshape(-1, 1))
huvi_zscore.partial_fit(huvi_range.reshape(-1, 1))
apparent_temp_zscore.partial_fit(temperature_range.reshape(-1, 1))
T_Min_zscore.partial_fit(temperature_range.reshape(-1, 1))
T_Max_zscore.partial_fit(temperature_range.reshape(-1, 1))
day_of_year_zscore.partial_fit(day_of_year_range.reshape(-1, 1))
data_week_zscore.partial_fit(week_range.reshape(-1, 1))
data_hour_zscore.partial_fit(hour_range.reshape(-1, 1))
data_minute_zscore.partial_fit(minute_range.reshape(-1, 1))
data_weekday_zscore.partial_fit(weekday_range.reshape(-1, 1))
minutes_of_the_day_zscore.partial_fit(minutes_of_the_day_range.reshape(-1, 1))
demand_zscore.partial_fit(demand_range.reshape(-1, 1))
# 部分計算 [E]
# 將部分計算得結果先做正規化 [B]
demand_dataframe['Temperature'] = temperature_zscore.fit_transform(temperature_reshape_data)
demand_dataframe['pres'] = pres_zscore.fit_transform(pres_reshape_data)
demand_dataframe['w_DSD'] = wind_speed_zscore.fit_transform(wind_speed_reshape_data)
demand_dataframe['w_DIR'] = wind_dir_zscore.fit_transform(wind_dir_reshape_data)
demand_dataframe['h_FX'] = max_wind_zscore.fit_transform(max_wind_reshape_data)
demand_dataframe['24R'] = r24_zscore.fit_transform(r24_reshape_data)
demand_dataframe['h_UVI'] = huvi_zscore.fit_transform(huvi_reshape_data)
demand_dataframe['apparent_temp'] = apparent_temp_zscore.fit_transform(apparent_temp_reshape_data)

demand_dataframe['T_Min'] = T_Min_zscore.fit_transform(T_Min_reshape_data)
demand_dataframe['T_Max'] = T_Max_zscore.fit_transform(T_Max_reshape_data)
demand_dataframe['day_of_year'] = day_of_year_zscore.fit_transform(day_of_year_reshape_data)
demand_dataframe['data_week'] = data_week_zscore.fit_transform(data_week_reshape_data)
demand_dataframe['data_hour'] = data_hour_zscore.fit_transform(data_hour_reshape_data)
demand_dataframe['data_minute'] = data_minute_zscore.fit_transform(data_minute_reshape_data)
demand_dataframe['data_weekday'] = data_weekday_zscore.fit_transform(data_weekday_reshape_data)
demand_dataframe['minutes_of_the_day'] = minutes_of_the_day_zscore.fit_transform(minutes_of_the_day_reshape_data)
demand_dataframe['demand_quarter_1'] = demand_zscore.fit_transform(demand_reshape_data)
# 將部分計算得結果先做正規化 [E]
# 資料正規化 [E]

In [ ]:
# 將資料分為訓練集與測試集 [B]
selected_column = ['Temperature', 'humd', 'pres', 'w_DSD', 'w_DIR', 'h_FX', '24R', 'h_UVI',
                   'T_Min', 'T_Max', 'day_of_year', 'data_week', 'data_hour',
                   'data_minute', 'minutes_of_the_day', 'apparent_temp', 'data_weekday']
all_train_features = demand_dataframe.loc[train_begin_date:train_end_date, selected_column]
all_train_labels = demand_dataframe.loc[train_begin_date:train_end_date, 'demand_quarter_1']

all_test_features = demand_dataframe.loc[test_begin_date:test_end_date, selected_column]
all_test_labels = demand_dataframe.loc[test_begin_date:test_end_date, 'demand_quarter_1']

all_predict_features = demand_dataframe.loc[predict_begin_date:predict_end_date, selected_column]
all_predict_labels = demand_dataframe.loc[predict_begin_date:predict_end_date, 'demand_quarter_1']
# 將資料分為訓練集與測試集 [E]

In [ ]:
# 將 dataframe 轉為 np 陣列 [B]
all_train_features = np.array(all_train_features)
all_test_features = np.array(all_test_features)
all_predict_features = np.array(all_predict_features)
# origin_data = np.array(list(data["hospitalizations_num"]))
# features = np.array([
#                 list(data['dayweight']),
#                 list(data['MAX_PM2_5']),
#                 list(data['hPa']),
#                 list(data['temperature']),
#                 list(data['relative_humidity']),
#                 list(data['wind_speed']),
#                 list(data['precipitation']),
#             ])
# 將 dataframe 轉為 np 陣列 [E]

In [ ]:
# 使用 AUTO ARIMA 來讓模型自動找出 p、d、q，並訓練模型 [B]
model = pm.auto_arima(all_train_labels, X=all_train_features, start_p=0, d=1, start_q=0, max_p=5, max_d=5, max_q=5, start_P=0,
                       D=1, start_Q=0, max_P=5, max_D=5, max_Q=5, m=12, seasonal=True,
                       error_action='warn', trace=True, suppress_warnings=True, stepwise=True, random_state=20,
                       n_fits=30)

print(model.summary())
# 使用 AUTO ARIMA 來讓模型自動找出 p、d、q，並訓練模型 [E]

In [ ]:
# 使用 SARIMAX 來讓模型自動找出 p、d、q [B]
arima2 = SARIMAX(y_train, order=(1, 1, 3))
# 使用 SARIMAX 來讓模型自動找出 p、d、q [E]

In [ ]:
# 用訓練好的模型做預測 [B]
res = arima2.fit(disp=False)
print(res.summary())
# 用訓練好的模型做預測 [E]

------------

In [ ]:
arima1 = SARIMAX(y_train, order=(10, 0, 3))

In [ ]:
res = arima1.fit(disp=False)
print(res.summary())

In [ ]:
arima3 = SARIMAX(y_train, order=(10, 0, 10), initialization='approximate_diffuse')

In [ ]:
res = arima3.fit(disp=False)
print(res.summary())